# Calculating state-by-state implied infection numbers

This notebook tries to compute what the full infection numbers in the past and present likely were/are.

It does so in the past by blending variables for "median days from infection to death" and "infection fatility rate" (IFR) with smoothed death rates. In other words, days_to_death days before date D, there must have been roughly (deaths_on_date_D / IFR) infections to end up with a given number of deaths on date D.

It does in the present to looking at what percentage of infections were confirmed on the last day calculated in the past, and applying that percentage to the new infections found since then. That doesn't quite take into account if there is a significant ramping of testing during that time, but it should be close enough.

The principal source of death data is files from the NY Times, supplemented by a more accurate DateOfDeath.csv from Massachusetts. The source of testing data is The COVID Tracking Project, maintained by The Atlantic.

NOTE: Prior to running this notebook, you should retrieve the latest DateOfDeath.csv file by:

1. going to https://www.mass.gov/info-details/covid-19-response-reporting,
2. downloading the raw data zip from the line saying "Raw data used to create the dashboard is available here:"
3. copying the DateofDeath.csv in that file to the same directory as the notebook

Yeah, that could be automated. Just haven't done it yet...

In [ ]:
%matplotlib inline
import numpy
import pandas
import matplotlib
import matplotlib.pyplot as plt

from common2 import load_data, smooth_series, calc_mid_weekly_average
from common2 import calc_state_stats, get_infections_df, find_smooth_dates

In [ ]:
# Earliest date that there is sufficient data for all states, including MA
EARLIEST_DATE = pandas.Period('2020-03-10', freq='D')
LATEST_DATE = pandas.Period('2020-08-11', freq='D')
LATEST_DATE = None

In [ ]:
latest_date, meta, nyt_stats, ct_stats = load_data(EARLIEST_DATE, LATEST_DATE)
print(f"Latest date = {str(latest_date)}")

In [ ]:
nyt_stats.tail(2)

In [ ]:
ct_stats.tail()

In [ ]:
# raise ValueError()

### Put the two datasets together

In [ ]:
ct1 = ct_stats.set_index(['ST', 'Date']).sort_index()[['Pos', 'Neg']]
nyt1 = nyt_stats.set_index(['ST', 'Date']).sort_index()[['Deaths']]
both = ct1.join(nyt1)
meta_tmp = meta.set_index('ST')

In [ ]:
states = [calc_state_stats(state, df, meta_tmp, latest_date)
          for state, df in both.reset_index().groupby('ST')]
states[-17].tail(2)

In [ ]:
stats = pandas.concat(states).reset_index()
stats[stats.ST == 'WV'].tail(20)[['Date', 'RawDeaths', 'Deaths', 'Deaths7']]

### Calculate new stats, state by state

In [ ]:
IFR_S, IFR_E = 0.011, 0.0035
IFR_S_S, IFR_E_S = f'{100*IFR_S:.1f}%', f'{100*IFR_E:.2f}%', 
infected_states = get_infections_df(states, 19, IFR_S, IFR_E, 4, 10)
print(infected_states.NewInf.sum())
infected_states.tail(3)

In [ ]:
fam = infected_states.reset_index()[['Date', 'NewInf']].groupby('Date').sum().plot(
    title=f"Infection Estimations, 19 median days to death, "
          f"IFR improving {IFR_S_S} - {IFR_E_S}",
    figsize=(13,5), legend=None, ylim=0
)

In [ ]:
fam = infected_states.reset_index()[['Date', 'Deaths7']].groupby('Date').sum().plot(
    title="Deaths", figsize=(10,4),
    legend=None, ylim=0
)

In [ ]:
foozle = infected_states.reset_index()[['Date', 'NewInf', 'Deaths7']].groupby('Date').sum()
foozle.columns = ['Infections', 'Deaths']
ax = foozle.plot(
    title=f"Daily Infections vs. Deaths, 19 median days to death, "
          f"IFR improving {IFR_S_S} - {IFR_E_S}",
    secondary_y='Deaths', figsize=(13,5), ylim=0)

In [ ]:
foo = infected_states.reset_index().set_index(['Date', 'ST']).sort_index()
foo = foo[['Pop', 'Confirms7', 'Deaths7', 'NewInf', 'AIPer1000', 'AUPer1000', 'PctFound']]
faz = foo.loc[latest_date, :].sort_values('AUPer1000', ascending=False).copy()
faz = faz.reset_index()[['ST', 'Pop', 'Confirms7', 'Deaths7', 'AIPer1000', 'AUPer1000', 'PctFound']]
faz.columns = ['ST', 'Pop', 'Cases', 'Deaths', 'AIPer1000', 'ActUnk1000', 'PctFound']
faz = faz.set_index('ST')
faz

In [ ]:
faz[faz.PctFound <= 0.25].sort_values('PctFound')

In [ ]:
fam = infected_states[['Pop', 'Confirms7', 'Deaths7', 'NewInf']].copy()
fam['C7Per'] = fam.Confirms7 / fam.Pop
fam['D7Per'] = fam.Deaths7 / fam.Pop
fam['NIPer'] = fam.NewInf / fam.Pop
fam = fam.reset_index()[['ST', 'NIPer', 'C7Per', 'D7Per']]
fam.columns = ['ST', 'Infections', 'Confirms', 'Deaths']
fam = fam.groupby('ST').max().copy()
print("Maximum deaths/M/day states ever had")
fam.sort_values('Deaths', ascending=False)

In [ ]:
raise ValueError()

## Now for the charts...

In [ ]:
infected_states.loc['NE', :].tail(30)

In [ ]:
st_names = ['NY', 'NJ', 'MA']
st_names = ['GA']
st_names = list(infected_states.index.get_level_values(0).unique())
st_names = ['AL', 'AR', 'FL', 'GA', 'HI', 'IA', 'KY',
            'MO', 'NC', 'ND', 'OK', 'SC', 'SD', 'TN', 'VA', ]
num_plots = max(len(st_names), 2)
fig, axes = plt.subplots(num_plots, figsize=(10, 4*num_plots))
for i, state in enumerate(st_names):
    try:
        infected_states.loc[state, :].NIPerM.plot(ax=axes[i], title=state)
    except:
        pass


In [ ]:
infected_states.index.get_level_values(0).unique()

In [ ]:
df = pandas.concat(states)[['DTests7']].reset_index()
st_names = list(df.ST.unique())
fig, axes = plt.subplots(len(st_names), figsize=(10, 4*len(st_names)))
for i, state in enumerate(st_names):
    try:
        df[df.ST == state].set_index('Date').DTests7.plot(ax=axes[i], title=state)
    except:
        pass


In [ ]:
st = 'MA'
# _ = infected_states.loc[st, :].NIPerM.plot(title=st, legend=True)
_ = infected_states.loc[st, :].DPerM.plot(title=st, secondary_y='DPerM', legend=True)

In [ ]:
infected_states.loc['DE', :].loc['2020-06-15':'2020-06-16', :]

In [ ]:
st_names = list(infected_states.index.get_level_values(0).unique())

In [ ]:
for st in st_names:
    fam = infected_states.loc[st, :].reset_index()[['Date', 'RawInc']].groupby('Date').sum().plot(
        title=f"{st}",
        figsize=(13,5), legend=None, ylim=0
    )

In [ ]:
infected_states.loc['WV', :].tail(20)

In [ ]:
states[-2].tail(20)

In [ ]:
ct_stats[ct_stats.ST == 'WV'].tail(20)

In [ ]:
raw = pandas.read_csv('https://covidtracking.com/api/v1/states/daily.csv')
raw.tail(1)

In [ ]:
foo = raw[raw.state == 'WV'].copy()
foo.date = [pandas.Period(str(v)) for v in foo.date]
foo = foo[['date', 'state', 'positive', 'negative']]
foo.columns = ['Date', 'ST', 'Pos', 'Neg']
foo = foo.set_index(['ST', 'Date'])
foo.Pos = smooth_series(foo.Pos)
foo.Neg = smooth_series(foo.Neg)
foo['Tests'] = foo.Pos + foo.Neg
foo.head(20)

In [ ]:
ct_stats.tail(20)